In [1]:
import os, sys

# Move one level up to Source Code
os.chdir(r"C:\Users\husseien\Desktop\340915149_322754953\Source Code")

# Optional: make sure Python can see this folder
sys.path.insert(0, os.getcwd())

print("CWD now:", os.getcwd())
print("sys.path[0]:", sys.path[0])
import pandas as pd
import os
import torch
import os, sys

# Add Source Code folder to sys.path
BASE_DIR = r"C:\Users\husseien\Desktop\340915149_322754953\Source Code"

if BASE_DIR not in sys.path:
    sys.path.insert(0, BASE_DIR)

# Optional: change working directory (recommended)
if os.getcwd() != BASE_DIR:
    os.chdir(BASE_DIR)

# Now you can import setup_paths
import setup_paths  # this will run the code in setup_paths.py


import os

# Base data folder
DATA_DIR = r"C:\Users\husseien\Desktop\340915149_322754953\Source Code\data"

# Path to labeled train.csv
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")

# Path to folder containing all raw CSV files
FILES_DIRECTORY = os.path.join(DATA_DIR, "unlabeled", "unlabeled")

# Check
print("TRAIN_CSV exists:", os.path.exists(TRAIN_CSV))
print("FILES_DIRECTORY exists:", os.path.exists(FILES_DIRECTORY))
print("Example CSVs:", os.listdir(FILES_DIRECTORY)[:5])



from models_utils.GLOBALS import files_directory
from RF_XGB.RandomForest import get_rf_data
from models_utils.utils import convert_to_features
from models_utils.Datasets import *
from models_utils.GLOBALS import *
import torch.nn.functional as F
from CNN.CNN import MultivariateCNN
from CNN.cnn_utils import train_cnn, get_train_data

CWD now: C:\Users\husseien\Desktop\340915149_322754953\Source Code
sys.path[0]: C:\Users\husseien\Desktop\340915149_322754953\Source Code
CWD set to C:\Users\husseien\Desktop\340915149_322754953\Source Code
TRAIN_CSV exists: True
FILES_DIRECTORY exists: True
Example CSVs: ['0.csv', '1.csv', '10.csv', '100.csv', '1000.csv']


In [2]:
# get train data
train_data = pd.read_csv(
    r"C:\Users\husseien\Desktop\340915149_322754953\Source Code\data\train.csv",
    index_col=0
)
train_data['activity'] = train_data['activity'].map(activity_id_mapping)
data_type_1 = train_data[train_data['sensor'] == 'vicon'].reset_index()
data_type_2 = train_data[train_data['sensor'] == 'smartwatch'].reset_index()

In [3]:
# sizes of padding/cutting
target_size_type1 = 3000
target_size_type2 = 1169

In [4]:
# train or load models
train_or_load_autoencoders = 'load'
if train_or_load_autoencoders == 'train':
    model_CNN_type1 = train_cnn('Type1OnlyCNN', data_type_1, '1', target_size_type1, 30, 64, 0.001)
    model_CNN_type2 = train_cnn('Type2OnlyCNN', data_type_2, '2', target_size_type2, 20, 64, 0.001)
elif train_or_load_autoencoders == 'load':
    model_CNN_type1 = MultivariateCNN(3, target_size_type1, 18).to(device)
    model_CNN_type1.load_state_dict(torch.load('Type1OnlyCNN.pth'))
    model_CNN_type2 = MultivariateCNN(3, target_size_type2, 18).to(device)
    model_CNN_type2.load_state_dict(torch.load(f'Type2OnlyCNN.pth'))
else:
    raise ValueError('Wrong train or load')

In [5]:
import pandas as pd
import torch
from CNN.CNN import MultivariateCNN
from CNN.cnn_utils import get_train_data

train_df = pd.read_csv(TRAIN_CSV)

type1_max_len = 0
type2_max_len = 0

for idx, row in train_df.iterrows():
    file_path = f"{files_directory}/{row['id']}.csv"
    data = pd.read_csv(file_path)

    if data.shape[1] == 3:
        type1_max_len = max(type1_max_len, len(data))
    else:
        acc_data = data[data.iloc[:, 0] == 'acceleration [m/s/s]'].iloc[:, 1:]
        type2_max_len = max(type2_max_len, len(acc_data))

print("Dynamic target sizes:", type1_max_len, type2_max_len)

model_CNN_type1 = MultivariateCNN(
    num_channels=3,
    input_length=type1_max_len,
    num_classes=18
).to(device)

model_CNN_type2 = MultivariateCNN(
    num_channels=3,
    input_length=type2_max_len,
    num_classes=18
).to(device)

calculate_or_load_train_data = 'load'

if calculate_or_load_train_data == 'calculate':
    data_type_1, data_type_2 = get_train_data(
        savename='train_Only_CNN',
        modelType1=model_CNN_type1,
        modelType2=model_CNN_type2,
        target_size_type1=type1_max_len,
        target_size_type2=type2_max_len,
        embedding_size=18,
        is_autoencoder=False
    )
elif calculate_or_load_train_data == 'load':
    data_type_1 = pd.read_csv('train_Only_CNN_type1.csv')
    data_type_2 = pd.read_csv('train_Only_CNN_type2.csv')


Dynamic target sizes: 4000 1350


In [ ]:
# get and save results for test data
results_list = []
sample_submission_path = os.path.join(DATA_DIR, "sample_submission.csv")

for i, file_id in enumerate(pd.read_csv(sample_submission_path)['sample_id'].to_list()):

    class_path = os.path.join(files_directory, f"{file_id}.csv")
    new_data = pd.read_csv(class_path)

    if new_data.shape[1] == 3:
        if len(new_data) < 4000:
            new_data = pad_sequence(new_data, 4000)
        new_data = torch.tensor(new_data.values, dtype=torch.float32).to(device)
        normalized_new_data = (new_data - min_values_type1) / (max_values_type1 - min_values_type1 + 1e-6)
        normalized_new_data = normalized_new_data.transpose(0, 1)
        logits = model_CNN_type1(normalized_new_data)


    else:
        new_data = new_data[new_data.iloc[:, 0] == 'acceleration [m/s/s]'].iloc[:, 1:]
        if len(new_data) < 1350:
            new_data = pad_sequence(new_data, 1350)
        new_data = torch.tensor(new_data.values, dtype=torch.float32).to(device)
        normalized_new_data = (new_data - min_values_type2) / (max_values_type2 - min_values_type2 + 1e-6)
        normalized_new_data = normalized_new_data.transpose(0, 1)

        logits = model_CNN_type2(normalized_new_data)

    predictions = F.softmax(logits, dim=1)
    res_dict = {activity: predictions.squeeze()[id].item() for id, activity in id_activity_mapping.items()}

    result_dict = {label: res_dict.get(label, 0) for label in activity_id_mapping.keys()}
    result_dict['sample_id'] = file_id
    results_list.append(result_dict)
results = pd.DataFrame(results_list, columns=['sample_id'] + list(activity_id_mapping.keys()))
results.fillna(0).to_csv('results_raw_cnn.csv', index=False)

In [ ]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load results CSV
results = pd.read_csv('results_raw_cnn.csv')

# Suppose you have true labels for train/val in train_df
# Map 'activity' to one-hot for comparison
activity_cols = [c for c in results.columns if c != 'sample_id']
true_labels = train_df.set_index('id')['activity'].map(activity_id_mapping)  # numeric mapping
true_labels_onehot = pd.get_dummies(true_labels)

# Merge predicted probabilities with true labels
# Only keep samples that exist in results
common_ids = results['sample_id'].isin(true_labels_onehot.index)
merged_results = results[common_ids].copy()
merged_results['true_activity'] = [true_labels_onehot.loc[i].idxmax() for i in merged_results['sample_id']]

# Calculate predicted class
merged_results['predicted_activity'] = merged_results[activity_cols].idxmax(axis=1)

# Identify good vs bad predictions
merged_results['correct'] = merged_results['predicted_activity'] == merged_results['true_activity']

# 1. Plot examples of good predictions
good_preds = merged_results[merged_results['correct']].sample(min(5, len(merged_results[merged_results['correct']])))

for _, row in good_preds.iterrows():
    probs = row[activity_cols].values
    top_classes = [activity_cols[i] for i in np.argsort(probs)[-5:][::-1]]
    top_probs = np.sort(probs)[-5:][::-1]
    plt.figure(figsize=(6,4))
    sns.barplot(x=top_classes, y=top_probs, palette='Greens')
    plt.title(f"GOOD Prediction: True={row['true_activity']}, Predicted={row['predicted_activity']}")
    plt.ylabel("Probability")
    plt.ylim(0,1)
    plt.show()

# 2. Plot examples of bad predictions
bad_preds = merged_results[~merged_results['correct']].sample(min(5, len(merged_results[~merged_results['correct']])))
for _, row in bad_preds.iterrows():
    probs = row[activity_cols].values
    top_classes = [activity_cols[i] for i in np.argsort(probs)[-5:][::-1]]
    top_probs = np.sort(probs)[-5:][::-1]
    plt.figure(figsize=(6,4))
    sns.barplot(x=top_classes, y=top_probs, palette='Reds')
    plt.title(f"BAD Prediction: True={row['true_activity']}, Predicted={row['predicted_activity']}")
    plt.ylabel("Probability")
    plt.ylim(0,1)
    plt.show()

# 3. Optionally, show model uncertainty: max probability histogram
max_probs = merged_results[activity_cols].max(axis=1)
plt.figure(figsize=(8,4))
sns.histplot(max_probs, bins=20, kde=True)
plt.title("Model confidence for all samples")
plt.xlabel("Max probability")
plt.ylabel("Number of samples")
plt.show()
